In [1]:
!pip install -q --upgrade pip
!pip install -q ortools tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.


In [2]:
from typing import List, Tuple, Dict
from ortools.sat.python import cp_model
from tqdm.auto import tqdm
import json
import os

def norm(points):
    min_x = min(p[0] for p in points)
    min_y = min(p[1] for p in points)
    normed = sorted((p[0] - min_x, p[1] - min_y) for p in points)
    return tuple(normed)

def rot(points):
    return set((y, -x) for (x, y) in points)

def flip(points):
    return set((-x, y) for (x, y) in points)

class Shape:
    def __init__(self, grid: List[str]):
        self.points = []
        for y in range(len(grid)):
            for x in range(len(grid[0])):
                if grid[y][x] == '#':
                    self.points.append((x, y))

    def generate_variants(self):
        seen = set()
        res = []
        base = set(self.points)
        for f in [False, True]:
            points = flip(base) if f else base
            for _ in range(4):
                points = rot(points)
                normed = norm(points)
                if normed not in seen:
                    seen.add(normed)
                    res.append(normed)
        return res

def read_input(path: str):
    with open(path, 'r') as f:
        lines = f.readlines()
    shapes = []
    for i in range(6):
        shape = []
        for line in lines[i*5+1:i*5+4]:
            shape.append(line.strip())
        shapes.append(Shape(shape))
    queries = []
    for line in lines[30:]:
        if not line.strip():
            continue
        left, right = line.strip().split(': ')
        w, h = map(int, left.split('x'))
        lst = list(map(int, right.split()))
        queries.append((w, h, lst))
    return shapes, queries

def gen_placements(W, H, variants):
    placements = []
    for var in variants:
        max_x = max(dx for dx, dy in var)
        max_y = max(dy for dx, dy in var)
        for x0 in range(W - (max_x + 1) + 1):
            for y0 in range(H - (max_y + 1) + 1):
                cells = []
                ok = True
                for dx, dy in var:
                    x, y = x0 + dx, y0 + dy
                    if not (0 <= x < W and 0 <= y < H):
                        ok = False
                        break
                    cells.append(y * W + x)
                if ok:
                    placements.append(tuple(sorted(cells)))
    placements = list(set(placements))
    placements.sort()
    return placements

def can_fit_region(W, H, shape_variants, areas, target, time_limit=100.0, workers=8):
    need_area = 0
    for i in range(6):
        need_area += target[i] * areas[i]
    if need_area > W * H:
        return False
    if need_area == 0:
        return True

    placements_by_shape = [None] * 6
    for i in range(6):
        if target[i] == 0:
            placements_by_shape[i] = []
            continue
        pls = gen_placements(W, H, shape_variants[i])
        if not pls:
            return False
        placements_by_shape[i] = pls

    idxs = [i for i in range(6) if target[i] > 0]
    idxs.sort(key=lambda i: len(placements_by_shape[i]) / max(1, target[i]))

    model = cp_model.CpModel()
    cell_vars: List[List[cp_model.IntVar]] = [[] for _ in range(W * H)]
    shape_vars: List[List[cp_model.IntVar]] = [[] for _ in range(6)]

    for i in idxs:
        for cells in placements_by_shape[i]:
            v = model.NewBoolVar(f"s{i}_{len(shape_vars[i])}")
            shape_vars[i].append(v)
            for c in cells:
                cell_vars[c].append(v)

    for i in idxs:
        if not shape_vars[i]:
            return False
        model.Add(sum(shape_vars[i]) == target[i])

    for c in range(W * H):
        if cell_vars[c]:
            model.Add(sum(cell_vars[c]) <= 1)

    anchor_shape = idxs[0]
    if shape_vars[anchor_shape]:
        model.Add(shape_vars[anchor_shape][0] == 1)

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    solver.parameters.num_search_workers = workers
    solver.parameters.cp_model_presolve = True
    solver.parameters.linearization_level = 2
    res = solver.Solve(model)
    return res == cp_model.OPTIMAL or res == cp_model.FEASIBLE

def load_done(result_path: str):
    done = {}
    ok_count = 0
    if not os.path.exists(result_path):
        return done, ok_count
    with open(result_path, "r") as f:
        for line in f:
            if not line.strip():
                continue
            obj = json.loads(line)
            done[obj["idx"]] = obj["ok"]
            if obj["ok"]:
                ok_count += 1
    return done, ok_count

def solve_with_checkpoint(shapes: List[Shape], queries: List[Tuple[int, int, List[int]]],
                          result_path: str, count_path: str,
                          time_limit=100.0, workers=8) -> int:
    shape_variants = [shape.generate_variants() for shape in shapes]
    areas = [len(shape_variants[i][0]) for i in range(6)]

    done, ok_count = load_done(result_path)

    bar = tqdm(range(len(queries)), desc="Checking regions")
    for idx in bar:
        if idx in done:
            continue
        w, h, lst = queries[idx]
        bar.set_postfix({"size": f"{w}x{h}", "need": sum(lst), "ok": ok_count})

        ok = can_fit_region(w, h, shape_variants, areas, lst, time_limit=time_limit, workers=workers)

        with open(result_path, "a") as f:
            f.write(json.dumps({
                "idx": idx,
                "w": w,
                "h": h,
                "target": lst,
                "ok": bool(ok),
            }) + "\n")

        done[idx] = bool(ok)
        if ok:
            ok_count += 1

        with open(count_path, "w") as f:
            f.write(str(ok_count) + "\n")

    return ok_count


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_path = "/content/drive/MyDrive/aoc/day12.txt"
result_path = "/content/drive/MyDrive/aoc/day12_results.jsonl"
count_path = "/content/drive/MyDrive/aoc/day12_count.txt"

shapes, queries = read_input(input_path)

ans = solve_with_checkpoint(
    shapes, queries,
    result_path=result_path,
    count_path=count_path,
    time_limit=200.0,
    workers=8
)

print("done:", ans)

Checking regions:   0%|          | 0/1000 [00:00<?, ?it/s]